# Open Ocean
# Open Earth Fundation

<h1> Step 2: calculate different metrics for each modulating factor </h1>

This notebook is the second part of the `Step1_Curate_IUCN_RedList.ipynb`

<h2> Modulating Factor 2: Species Richness </h2>

Idea: Use the total number of species within the area of interest to get the base unit for a base credit.

### Data needed for this project

At this moment we have not identified a database on marine biodiversity for the analysis of this modulating factor, however we make several assumptions based on literature and some national inventories of species.

### 1. Importing libraries.

In [ ]:
import os
import boto3

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point

In [ ]:
import fiona; #help(fiona.open)

OEF Functions

In [ ]:
from MBU_utils import *

### 2. Load data
 (....)

### 3. Preliminary calculations

Info from SINAC, link to [Google Drive](https://drive.google.com/file/d/1Bztf55bKgZi7X9nJPbWGdOUtW7Zv0-3_/view?usp=sharing) document
```
The CINP is home to 1688 species of marine fauna, of which
45 are endemic. This represents 47,4 % of the endemism in
Costa Rica. In other words, the CINP contributes half of the
country’s unique biological richness.
```
SINAC (Sistema Nacional de Áreas de Conservación). 2018. Importancia de los ecosistemas y las poblaciones del sitio de interés: Parque Nacional Isla del Coco (PNIC), Área Marina de Manejo Montes Submarinos (AMM MS) y aguas adyacentes. Área de Conservación Marina Cocos. San José, Costa Rica. 68 p.

Note: this was the old park
```
The CINP, created in 1978, is located 342 miles (500 km) southeast of Costa Rica’s Pacific coast and covers an area of 785 sq miles (2034 km2). The island comprises an area of about 9 sq miles (23 km2) and the marine area covers the remaining 776 sq miles (2011 km2). The SMMA was created in 2011 and was the first protected wildlife area in this category in Costa Rica. It encompasses 3725 sq miles (9649 km2) of the Pacific Ocean waters around the CINP (Figure 1).
```

Note: World Register of Marine Species (WoRMS)  contains 243,000 species in its inventory.

https://en.wikipedia.org/wiki/Marine_life#/media/File:Marine_animal_biodiversity.png shows 201,004.

https://en.wikipedia.org/wiki/Species_richness shows species richness in terms of km? or per sq km?

https://www.researchgate.net/figure/Global-marine-species-richness-for-all-taxa-Source-Titterson-et-al-2010_fig2_305636318
this is at a coarse resolution, but we can use this to normalize marine richness?

According to: https://www.researchgate.net/publication/45406822_Global_patterns_and_predictors_of_marine_biodiversity_across_taxa
we have: 
11,567 total - 60 seagrasses - 32 mangroves - 39 foraminifera

**Marine biodiversity Assumption 1**

Assuming a total number of species of marine fauna of $N_\text{total}$ (here `N_species_fauna_total`)

In [ ]:
N_species_fauna_total = 11567 - 60 - 32 - 39
print(N_species_fauna_total)

**Marine biodiversity Assumption 2**

Assuming a number of species of marine fauna within the area of interest of $N_\text{local}$ (here `N_species_fauna`). This is extrapolating the number of species given by SINAC to the entire ACMC area.

In [ ]:
N_species_fauna = 1688

**Marine biodiversity Assumption 3**

The marine fauna is uniformely distributed accross the ACMC... here because we don't have:
- $N_\text{local}$(square kilometer 1)
- $N_\text{local}$(square kilometer 2)
- ...
- $N_\text{local}$(square kilometer  N)

Thus our "base unit" for credits is homogeneous throughout ACMC. 

Note 1: this is for illustrative purposes and this is a WRONG assumption! but we do not have a heat map of marine fauna!

Note 2: another approach can be that the representativeness of biodiversity in ACMC with respect to the world total. Value that can be penalized for the lack of data in ACMC.

In [ ]:
N_units_base = np.exp(N_species_fauna/N_species_fauna_total)
print(N_units_base)

Assuming we had the maximum species richness of $N_\text{total}$ = 2,555 (here `N_species_fauna_total`):

In [ ]:
np.exp(2555/N_species_fauna_total)

That's the "maximum base unit" per this logic.

Assuming we have very little species richness, e.g. $N_\text{total}$ = 10 (here `N_species_fauna_total` means there are only 10 species of fish, crabs, whatever):

In [ ]:
np.exp(10/N_species_fauna_total)

That's a lower bound for the "base unit" per this logic.

It is a very narrow range that does not reward high biodiversity this much. 

Is it "good" or "bad"? 

Note: 

Well, the advantages are as follows:
   * it minimizes the impact of measurements errors and uncertainty. Given that measuring marine biodiversity is difficult, this may actually be a good thing.
    
   * it increases the influence of modulating factors, such as:
        - vulnerable species
        - endemism
        - habitat type
        - etc.

### Another way

**1. Using IUCN Red List**

With polygons of distribution per each species

In [ ]:
ACMC = gpd.read_file('https://ocean-program.s3.amazonaws.com/data/raw/MPAs/ACMC.geojson')

In [ ]:
#Grid geo Dataframe
grid = create_grid(ACMC, 'hexagon', 1)

In [ ]:
%%time
df = gpd.read_file('/Users/maureenfonseca/Desktop/Data-Oceans/ACMC_IUCN_data/gdf_ACMC_IUCN_range_status_filtered.shp') 

In [ ]:
#Join in a gdf all the geometries within ACMC
df2 = gpd.clip(df.set_crs(epsg=4326, allow_override=True), ACMC)

In [ ]:
df2 = df2[0:100]

In [ ]:
#Count the number of overlapping geometries 
overlap = count_overlapping_geometries(df2)

In [ ]:
#To count how many geometries are in each grid
merged = gpd.sjoin(overlap, grid, how='left')
merged['n_species']= overlap['count_intersections']

In [ ]:
# Compute stats per grid cell
#aggfunc: assigne the max value of all the geometries that dissolve
dissolve = merged.dissolve(by="index_right", aggfunc={'n_species': 'max'})

In [ ]:
# put this into cell
grid.loc[dissolve.index,'n_species'] = dissolve.n_species.values

In [ ]:
#Convert area from degrees to square kilometers
#this case apply only for Central America
#https://epsg.io/31970
grid_gdf['area_sqkm'] = (grid_gdf.to_crs(crs=31970).area)*10**(-6)

In [ ]:
#Calculate the MBUS from the endemic MF
grid_gdf['mbu_endemism'] = grid_gdf['n_value']*grid_gdf['area_sqkm']